In [1]:
import pandas as pd
import numpy as np
from plotnine import *
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
time = "2024-12-31_13:33"
time = "2025-01-02_08:33"
time = "2025-01-03_06:00"
time = "2025-01-04_09:44"
time = "2025-01-06_17:52"


raw_df = pd.read_csv(f"/projects/genomic-ml/da2343/ml_project_2/unsupervised/kmeans/results/{time}_results.csv")
raw_df

,window,train_actual_return,train_num_trades,train_direction,test_actual_return,test_num_trades,test_direction,train_average_return,train_sharpe_ratio,train_profit_factor,...,total_windows,reverse_test,num_clusters,clustering_algorithm,train_period,test_period,random_seed,instrument,num_perceptually_important_points,price_history_length
0,301,0.212593,241,short,0.007682,25,long,0.205182,2.010645,1000.0,...,91,True,5,kmeans,7,1,623,GBP_CHF_M15,4,24
1,302,0.172106,221,short,-0.087007,48,long,0.205182,2.010645,1000.0,...,91,True,5,kmeans,7,1,623,GBP_CHF_M15,4,24
2,303,0.289080,264,short,-0.009291,44,long,0.205182,2.010645,1000.0,...,91,True,5,kmeans,7,1,623,GBP_CHF_M15,4,24
3,304,0.214637,201,short,0.022550,18,long,0.205182,2.010645,1000.0,...,91,True,5,kmeans,7,1,623,GBP_CHF_M15,4,24
4,305,0.163805,203,short,-0.012300,18,long,0.205182,2.010645,1000.0,...,91,True,5,kmeans,7,1,623,GBP_CHF_M15,4,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1498,397,0.234317,501,short,0.091215,39,long,0.222230,1.802169,1000.0,...,87,True,5,kmeans,10,1,594,AUD_NZD_M15,4,24
1499,398,0.039739,392,long,-0.002578,32,short,0.222230,1.802169,1000.0,...,87,True,5,kmeans,10,1,594,AUD_NZD_M15,4,24
1500,399,0.039943,392,long,0.006480,32,short,0.222230,1.802169,1000.0,...,87,True,5,kmeans,10,1,594,AUD_NZD_M15,4,24
1501,400,0.027183,392,long,-0.002554,32,short,0.222230,1.802169,1000.0,...,87,True,5,kmeans,10,1,594,AUD_NZD_M15,4,24


In [5]:
def analyze_performance(raw_df):
    # Group by relevant parameters and calculate averages
    group_cols = ['instrument', 'num_clusters', 'clustering_algorithm', 'train_period', 
                 'reverse_test', 'random_seed']
    
    # Calculate mean performance metrics for each parameter combination
    results_df = raw_df.groupby(group_cols).agg({
        'test_profit_factor': 'mean',
        'test_sharpe_ratio': 'mean',
        'test_win_ratio': 'mean',
        'test_num_trades': 'mean'
    }).reset_index()
    
    # Calculate combined score
    results_df['combined_score'] = (
        results_df['test_profit_factor'] * 0.4 +
        results_df['test_sharpe_ratio'] * 0.4 +
        (results_df['test_num_trades'] / results_df['test_num_trades'].max()) * 0.2
    )
    
    # Create display DataFrame with proper formatting
    display_df = pd.DataFrame({
        'Instrument': results_df['instrument'],
        'Clusters': results_df['num_clusters'],
        'Algorithm': results_df['clustering_algorithm'],
        'Train Period': results_df['train_period'].astype(str) + 'w',
        'Rev.Test': results_df['reverse_test'],
        'Seed': results_df['random_seed'],
        'PF': results_df['test_profit_factor'].round(3),
        'Sharpe': results_df['test_sharpe_ratio'].round(3),
        'Win%': (results_df['test_win_ratio'] * 100).round(1),
        'Trades': results_df['test_num_trades'].round(1),
        'Score': results_df['combined_score'].round(3)
    })
    
    # Sort by instrument name
    display_df = display_df.sort_values('Instrument')
    
    # Apply styling with color gradients
    styled_df = display_df.style\
        .format({
            'PF': '{:.3f}',
            'Sharpe': '{:.3f}',
            'Win%': '{:.1f}%',
            'Trades': '{:.1f}',
            'Score': '{:.3f}'
        })\
        .background_gradient(subset=['Score'], cmap='YlOrRd')\
        .background_gradient(subset=['PF'], cmap='YlOrRd')\
        .background_gradient(subset=['Sharpe'], cmap='YlOrRd')\
        .background_gradient(subset=['Win%'], cmap='YlOrRd')\
        .background_gradient(subset=['Trades'], cmap='YlOrRd')\
        .set_properties(**{
            'text-align': 'right',
            'font-family': 'monospace',
            'padding': '5px'
        })\
        .hide(axis="index")
    
    return results_df, styled_df

# Example usage:
results_df, styled_df = analyze_performance(raw_df)
styled_df  # Display the styled table

Instrument,Clusters,Algorithm,Train Period,Rev.Test,Seed,PF,Sharpe,Win%,Trades,Score
AUD_CAD_M15,5,birch,6w,False,42,0.910,-0.035,56.3%,32.0,0.516
AUD_JPY_M15,5,birch,9w,True,42,1.303,0.090,60.2%,30.3,0.714
AUD_NZD_M15,5,kmeans,10w,True,594,1.111,0.037,44.8%,38.6,0.659
AUD_USD_M15,5,kmeans,11w,True,640,1.128,0.045,47.5%,30.3,0.626
CAD_JPY_M15,5,birch,4w,False,42,1.252,0.087,50.5%,30.8,0.695
EUR_CAD_M15,5,kmeans,7w,True,673,1.122,0.043,54.6%,15.7,0.547
EUR_CHF_M15,5,birch,4w,False,42,1.216,0.067,45.5%,33.6,0.688
EUR_GBP_M15,5,kmeans,5w,True,279,1.975,0.228,57.1%,33.9,1.057
EUR_JPY_M15,5,kmeans,4w,True,886,1.077,0.026,50.5%,35.0,0.623
EUR_USD_M15,5,kmeans,4w,False,60,0.939,-0.021,46.0%,14.3,0.441


In [6]:
print(results_df)

     instrument  num_clusters clustering_algorithm  train_period  \
0   AUD_CAD_M15             5                birch             6   
1   AUD_JPY_M15             5                birch             9   
2   AUD_NZD_M15             5               kmeans            10   
3   AUD_USD_M15             5               kmeans            11   
4   CAD_JPY_M15             5                birch             4   
5   EUR_CAD_M15             5               kmeans             7   
6   EUR_CHF_M15             5                birch             4   
7   EUR_GBP_M15             5               kmeans             5   
8   EUR_JPY_M15             5               kmeans             4   
9   EUR_USD_M15             5               kmeans             4   
10  GBP_CHF_M15             5               kmeans             7   
11  GBP_JPY_M15             5                birch             9   
12  GBP_USD_M15             5               kmeans            11   
13  NZD_USD_M15             5               kmea